# Solo calculo jacobianos

soy una mentira, no automatizo casi nada

In [1]:
# Esta línea es una línea mágica de jupyter. Hace que puedas hacerle zoom y mover los grafiquitos. Si escribís directamente
# en un script de python hay que sacarla
%matplotlib ipympl

# Acá sí importamos las librerías como corresponde
import sympy as smp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm    # Le da color a los puntos de la superficie en base a su valor en z

In [3]:
# Defino la variable del tiempo y la masa
t, m, R = smp.symbols('t m R', real=True)

# Defino las variables cartesianas como las x_i y sus momentos como px_i
# después defino las variables de llegada como y_i y sus momentos como py_i
x1,x2,x3,px1,px2,px3 = smp.symbols(r'x_1 x_2 x_3 p_x1 p_x2 p_x3', real=True, cls=smp.Function)
y1,y2,y3,py1,py2,py3 = smp.symbols(r'y_1 y_2 y_3 p_y1 p_y2 p_y3', real=True, cls=smp.Function)

y1=y1(t); y2=y2(t); y3=y3(t)
py1=py1(t,y1,y2,y3); py2=py2(t,y1,y2,y3); py3=py3(t,y1,y2,y3)
dy1 = smp.diff(y1,t); dy2 = smp.diff(y2,t); dy3 = smp.diff(y3,t)

x1=x1(y1,y2,y3,py1,py2,py3); x2=x2(y1,y2,y3,py1,py2,py3); x3=x3(y1,y2,y3,py1,py2,py3)
px1=px1(y1,y2,y3,py1,py2,py3); px2=px2(y1,y2,y3,py1,py2,py3); px3=px3(y1,y2,y3,py1,py2,py3)


## Cambio de coordenadas en el espacio de fases para la esfera

In [288]:
# Ahora hay que definir la transformación de un sistema a otro
x1 = y1*smp.sin(y2)*smp.cos(y3)
x2 = y1*smp.sin(y2)*smp.sin(y3)
x3 = y1*smp.cos(y2)

# Calculo el lagrangiano
v2 = (smp.diff(x1,t)**2 + smp.diff(x2,t)**2 + smp.diff(x3,t)**2).simplify()
L = smp.Rational(1,2)*m*v2
L

m*(y_1(t)**2*sin(y_2(t))**2*Derivative(y_3(t), t)**2 + y_1(t)**2*Derivative(y_2(t), t)**2 + Derivative(y_1(t), t)**2)/2

In [289]:
# Calculo los momentos
px1 = (m*smp.diff(x1,t)).expand()
px2 = (m*smp.diff(x2,t)).expand()
px3 = (m*smp.diff(x3,t)).expand()
Px = smp.Matrix([px1,px2,px3])

py1 = smp.diff(L,dy1)
py2 = smp.diff(L,dy2)
py3 = smp.diff(L,dy3)
Py = smp.Matrix([py1,py2,py3])

In [257]:
Py

Matrix([
[                         m*Derivative(y_1(t), t)],
[               m*y_1(t)**2*Derivative(y_2(t), t)],
[m*y_1(t)**2*sin(y_2(t))**2*Derivative(y_3(t), t)]])

In [258]:
Px

Matrix([
[-m*y_1(t)*sin(y_2(t))*sin(y_3(t))*Derivative(y_3(t), t) + m*y_1(t)*cos(y_2(t))*cos(y_3(t))*Derivative(y_2(t), t) + m*sin(y_2(t))*cos(y_3(t))*Derivative(y_1(t), t)],
[ m*y_1(t)*sin(y_2(t))*cos(y_3(t))*Derivative(y_3(t), t) + m*y_1(t)*sin(y_3(t))*cos(y_2(t))*Derivative(y_2(t), t) + m*sin(y_2(t))*sin(y_3(t))*Derivative(y_1(t), t)],
[                                                                                 -m*y_1(t)*sin(y_2(t))*Derivative(y_2(t), t) + m*cos(y_2(t))*Derivative(y_1(t), t)]])

---

**Una vez calculados los momentos podemos mirar fuerte los vectores e identificar la relación entre los momentos.**

**Esto lo vamos a usar para redefinir a los momentos unos en función de otros.**

---

In [290]:
y1,y2,y3,py1,py2,py3 = smp.symbols(r'y_1 y_2 y_3 p_y1 p_y2 p_y3', real=True, cls=smp.Function)

y1=y1(t); y2=y2(t); y3=y3(t)
py1=py1(t,y1,y2,y3); py2=py2(t,y1,y2,y3); py3=py3(t,y1,y2,y3)

px1 = py1*smp.sin(y2)*smp.cos(y3) + py2* smp.cos(y2)*smp.cos(y3)/y1 + py3* smp.sin(y3)/(-y1*smp.sin(y2))
px2 = py1*smp.sin(y2)*smp.sin(y3) + py2* smp.cos(y2)*smp.sin(y3)/y1 + py3* smp.cos(y3)/(y1*smp.sin(y2))
px3 = py1*smp.cos(y2) + py2* smp.sin(y2)/(-y1)

In [296]:
# Defino una lista de variables
x_px = [x1,x2,x3,px1,px2,px3]
y_py = [y1,y2,y3,py1,py2,py3]

jacobiana = smp.Matrix([[smp.diff(x_px[i],y_py[j]) for j in range(6)] for i in range(6)]).applyfunc(lambda x: x.simplify())
jacobiana

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                          sin(y_2(t))*cos(y_3(t)),                                                                                                                                                                                                                                                                                                                                                                                                                              y_1(t)*cos(y_2(t))*cos(y_3(t)),                                                                                                               

In [298]:
jacobiana.det().simplify()

1

## Cambio de coordenadas en el espacio de fases para el toro

In [7]:
# Ahora hay que definir la transformación de un sistema a otro
x1 = (R + y1*smp.cos(y2))*smp.cos(y3)
x2 = (R + y1*smp.cos(y2))*smp.sin(y3)
x3 = y1*smp.sin(y2)

# Calculo el lagrangiano
v2 = (smp.diff(x1,t)**2 + smp.diff(x2,t)**2 + smp.diff(x3,t)**2).simplify()
L = (smp.Rational(1,2)*m*v2).simplify()
L

m*(R**2*Derivative(y_3(t), t)**2 + 2*R*y_1(t)*cos(y_2(t))*Derivative(y_3(t), t)**2 + y_1(t)**2*cos(y_2(t))**2*Derivative(y_3(t), t)**2 + y_1(t)**2*Derivative(y_2(t), t)**2 + Derivative(y_1(t), t)**2)/2

In [13]:
# Calculo los momentos
px1 = (m*smp.diff(x1,t)).expand()
px2 = (m*smp.diff(x2,t)).expand()
px3 = (m*smp.diff(x3,t)).expand()
Px = smp.Matrix([px1,px2,px3]).applyfunc(lambda x: x.simplify())

py1 = smp.diff(L,dy1)
py2 = smp.diff(L,dy2)
py3 = smp.diff(L,dy3)
Py = smp.Matrix([py1,py2,py3]).applyfunc(lambda x: x.simplify())

In [14]:
Py

Matrix([
[                            m*Derivative(y_1(t), t)],
[                  m*y_1(t)**2*Derivative(y_2(t), t)],
[m*(R + y_1(t)*cos(y_2(t)))**2*Derivative(y_3(t), t)]])

In [15]:
Px

Matrix([
[m*(-R*sin(y_3(t))*Derivative(y_3(t), t) - y_1(t)*sin(y_2(t))*cos(y_3(t))*Derivative(y_2(t), t) - y_1(t)*sin(y_3(t))*cos(y_2(t))*Derivative(y_3(t), t) + cos(y_2(t))*cos(y_3(t))*Derivative(y_1(t), t))],
[ m*(R*cos(y_3(t))*Derivative(y_3(t), t) - y_1(t)*sin(y_2(t))*sin(y_3(t))*Derivative(y_2(t), t) + y_1(t)*cos(y_2(t))*cos(y_3(t))*Derivative(y_3(t), t) + sin(y_3(t))*cos(y_2(t))*Derivative(y_1(t), t))],
[                                                                                                                      m*(y_1(t)*cos(y_2(t))*Derivative(y_2(t), t) + sin(y_2(t))*Derivative(y_1(t), t))]])

---

**Una vez calculados los momentos podemos mirar fuerte los vectores e identificar la relación entre los momentos.**

**Esto lo vamos a usar para redefinir a los momentos unos en función de otros.**

---

In [16]:
y1,y2,y3,py1,py2,py3 = smp.symbols(r'y_1 y_2 y_3 p_y1 p_y2 p_y3', real=True, cls=smp.Function)

y1=y1(t); y2=y2(t); y3=y3(t)
py1=py1(t,y1,y2,y3); py2=py2(t,y1,y2,y3); py3=py3(t,y1,y2,y3)

px1 = py1*smp.cos(y2)*smp.cos(y3) - py2* smp.sin(y2)*smp.cos(y3)/y1 - py3* smp.sin(y3)/(R + y1*smp.cos(y2))
px2 = py1*smp.cos(y2)*smp.sin(y3) - py2* smp.sin(y2)*smp.sin(y3)/y1 + py3* smp.cos(y3)/(R + y1*smp.cos(y2))
px3 = py1*smp.sin(y2) + py2* smp.cos(y2)/(-y1)

In [17]:
# Defino una lista de variables
x_px = [x1,x2,x3,px1,px2,px3]
y_py = [y1,y2,y3,py1,py2,py3]

jacobiana = smp.Matrix([[smp.diff(x_px[i],y_py[j]) for j in range(6)] for i in range(6)]).applyfunc(lambda x: x.simplify())
jacobiana

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     cos(y_2(t))*cos(y_3(t)),                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [18]:
jacobiana.det().simplify()

-cos(2*y_2(t))